In [1]:
!pip install ebaysdk

  Stored in directory: C:\Users\mojta\AppData\Local\pip\Cache\wheels\a5\76\f4\03967ae85aea44c3d93d8cbcc29f5c37da8e6c3d30694adb7c
Successfully built ebaysdk


In [2]:
MyToken = "AgAAAA**AQAAAA**aAAAAA**3+rCXA**nY+sHZ2PrBmdj6wVnY+sEZ2PrA2dj6ANloKlAZGDow2dj6x9nY+seQ**6voFAA**AAMAAA**y4HII3vHsCqPOFD5XxwoxNBZPrFz0chf4UdjnQQEYHhCuvyjrC5mGbttTLgx6LCwx9DwGig0yMlzavIuGCbYRzdqhAIA/PvSpqDjzdK8jRh1r8SIYFzvHP1sR5S9betmx8N4nJ7UvVUj5cDAHIqSkiQycQ4G66ADMQd9gWyRv39h0mVK+K+c0Z/Kkz7zMtROiiD4Y0dYfDuBCaJY6cTaTZtmnCBRy0kKJHnz9yCgnrWS6DCCmSuzNdLZq0NtOrA0NRslUQqHItrhqjTbBQfhq2NdJu1AteQmc25IkAkkSXjzRjOAkF7pGZhUOSMA2A6JjSOffSPIEKfYSorpC8CAiu8zeE3t4HqRPN4kbPbRKZ6//JXkMDhe7Aj5T+lU0UdE9l5ojtuH5nfm9af0ewuQHNFBTCTAT7gLneFJZSc6eAupZjfoktaVcNmsVVPGERIly9oOfspbQvMc+OHjWfHOyg2WXBzSioq2MhLhtDbOn7SUQdW2pVo27+xpwN7QZ4824oUr7tj8uy+P3VGFvLkfl2gQulg+H8NYty8uAxQN85YvSc5Vz3Id/vRIFPy4Sss3xNH619b0/XFCEZ9iJrxJYVLZw4Ictbbe2qYrbctrRehKlMtThEWZiF1N/uBB6iympClsuKh1OenarCieWOhjtL1oFG5NFD7cGoPLsMQeE337BhN6o6t90OBhqZLVMhCgovrDF0gSVW/bkYyGzquBUorJ3Oi+6vyXnGLTZorPqHC63aXwZRW81hqarLN0qhsa"
MyCertID='PRD-eaa024340831-68cd-4d05-80cc-37ae'
MyDevID='da60db7a-fb35-4b1d-af99-e1d74f695fd7'
MyAppID='SMojtaba-Analyzin-PRD-2eaa02434-919db739'
Entry_Per_Page  = 100
fridges_details = r"fridges_info"
data_dir = "data"
Daily_Call = 1160

In [3]:
import os
from datetime import datetime
import codecs
from ebaysdk.trading import Connection as Trading
import ebaysdk
from ebaysdk.finding import Connection as finding
from ebaysdk.shopping import Connection as Shopping
from bs4 import BeautifulSoup
import requests


In [4]:
def write_fridges_info(item, header=False):
        global fridges_details
        global data_dir
        if not header:
            f = codecs.open(data_dir + os.path.sep +fridges_details + datetime.now()
                        .strftime("-%Y-%m") + '.csv', "a", encoding="utf8")
            row_csv = "{0},{1},{2},{3},{4},{5},{6},{7},{8},{9},{10},{11},{12},{13}\r\n".format(
                item["itemId"],
                item["title"].replace(",", "^") if item["title"] is not None else item["title"],
                item['categoryName'].replace(",", "^") if item["categoryName"] is not None else item["categoryName"],
                item["location"].replace(",", "^") if item["location"] is not None else item["location"],
                item["country"],
                item["model"].replace(",", "^") if item["model"] is not None else item["model"],
                item["brand"].replace(",", "^") if item["brand"] is not None else item["brand"],
                item["capacity"],
                item["condition"],
                item["deliveryTime"].replace(",", "^") if item["deliveryTime"] is not None else item["deliveryTime"],
                item["dispatchTimeMax"],
                item["price"],
                item["currency"],
                item["viewItemURL"],
            )
        else:
            f = codecs.open(data_dir + os.path.sep +fridges_details + datetime.now()
                        .strftime("-%Y-%m") + '.csv', "w", encoding="utf8")
            row_csv = "{0},{1},{2},{3},{4},{5},{6},{7},{8},{9},{10},{11},{12},{13}\r\n".format("ID", "Title", "Category", "Location", "Country",
                                                                        "Model","Brand","Capacity","Condition","DeliveryTime","DispatchTimeMax","Price",
                                                                                          "Currency","URL")
        f.write(row_csv)
        f.close()
        
write_fridges_info({},header=True)

In [5]:
def issueMainQuery(pgn=1):
    global Daily_Call
    api = finding(siteid='EBAY-AU', appid='SMojtaba-Analyzin-PRD-2eaa02434-919db739',config_file=None)
    api.execute('findItemsAdvanced', {
        'keywords': '',
        'categoryId': ['20713', '71262'],
        'itemFilter': [
            {'name': 'Condition', 'value': ['Used','New']},

        ],

        'paginationInput': {
            'entriesPerPage': Entry_Per_Page,
            'pageNumber': pgn
        },
        'sortOrder': 'CurrentPriceHighest'
    })
    Daily_Call += 1
    return api.response.dict()

In [6]:
def getExtraInfoAPI(item):
    global Daily_Call
    Daily_Call +=1
    api = Trading(certid=MyCertID,devid=MyDevID,token=MyToken, appid=MyAppID, config_file=None)
    response = api.execute('GetItem', {'ItemID':item["itemId"], 'IncludeItemSpecifics' : True })
    dictstr = response.dict()
    
    Brand=None
    Capacity = None
    Model = None
    DeliveryTime=None
    DispatchTimeMax=None
    
    
    if "ShippingServiceOptions" in dictstr["Item"]["ShippingDetails"].keys():
        DeliveryTime = dictstr["Item"]["ShippingDetails"]["ShippingServiceOptions"]["ShippingTimeMax"]
    if "DispatchTimeMax" in dictstr["Item"].keys():
        DispatchTimeMax = dictstr["Item"]["DispatchTimeMax"]
    for v in dictstr["Item"]["ItemSpecifics"]["NameValueList"] :
        if v["Name"] == "Brand" :
            Brand = v["Value"]
        elif v["Name"] == "Model" :
            Model = v["Value"]
        elif v["Name"] == "Capacity" :
            Capacity = v["Value"]
    item["brand"] = Brand
    item["capacity"]  =Capacity
    item["model"] = Model
    item["deliveryTime"] = DeliveryTime
    item["dispatchTimeMax"] = DispatchTimeMax
    
    #display(dictstr)

In [7]:
def getExtraInfoBS4(item):
    page = requests.get(item["viewItemURL"])
    soup = BeautifulSoup(page.content, 'html.parser')
    item_specific = soup.select('div.itemAttr td')
    
    Brand=None
    Condition=None
    Capacity = None
    Model = None
    DeliveryTime=None
    DispatchTimeMax=None
    
    for idx,tag in enumerate(item_specific[::2]):  
        #print(idx,tag.text.strip())

#         if tag.text.strip().startswith("Condition"):
#             txt = item_specific[idx*2+1].text.strip()
#             if  txt.startswith("Brand New"):
#                 Condition ="Brand New"
#             elif txt.startswith("Used"):
#                 Condition ="Used"    
        if tag.text.strip().startswith("Brand") :
            Brand = item_specific[idx*2+1].text.strip()
        elif tag.text.strip().startswith("Model") :
            Model = item_specific[idx*2+1].text.strip()
        elif tag.text.strip().startswith("Capacity") :
            Capacity = item_specific[idx*2+1].text.strip()


    dl_time =  soup.select('span.vi-acc-del-range')
    if len(dl_time)>0:
        DeliveryTime = dl_time[0].text

    dispatch_time =  soup.select('span.sh-DlvryDtl')
    if len(dispatch_time)>0:
        DispatchTimeMax = dispatch_time[0].text[20:22]

    item["deliveryTime"] = DeliveryTime
    item["dispatchTimeMax"] = DispatchTimeMax
    item["brand"] = Brand
    item["capacity"]  =Capacity
    item["model"] = Model
    
    return

In [8]:
def getExtraInfo(item):
    global Daily_Call
    if Daily_Call >= 4999:
        getExtraInfoBS4(item)
    else: 
        try : 
            getExtraInfoAPI(item)
        except:
            getExtraInfoBS4(item)

In [9]:
def extractCapacity(item):
    tokens = item["title"].split()
    cap = 0 
    for t in tokens :
        if t.endswith("Lt") or t.endswith("lt") :
            try : 
                cap +=int(t[:-2])
            except:
                pass
        elif t.endswith("L"):   
            try : 
                cap +=int(t[:-1])
            except:
                pass
    if cap>0 :
        item["capacity"] = cap
    return    
        

In [ ]:
dictstr = issueMainQuery()
totalCount = int(dictstr["paginationOutput"]["totalEntries"])
print("Total Entries : {}".format(totalCount))
print("*"*30)
page_count = totalCount//100+1
item_list =[]
for pgn in range(1,page_count+1) :
    print("-"*30)
    print("Current Page : {}".format(pgn))
    itm_cnt = 1
    print("-"*30)
    dictstr = issueMainQuery(pgn)
    try : 
        for item in dictstr['searchResult']['item']:
            print("Current Item Index  : {}".format( (pgn-1)*Entry_Per_Page + itm_cnt))
            item_data={}
            print('ItemID: {}'.format(item['itemId']))
            item_data["itemId"] = item['itemId']
            print('Title: {}'.format(item['title']))
            item_data["title"] = item['title']
            print('CategoryID: {}'.format(item['primaryCategory']['categoryId']))
            print('CategoryName: {}'.format(item['primaryCategory']['categoryName']))
            item_data["categoryName"] = item['primaryCategory']['categoryName']
            print('Location: {}'.format(item['location']))
            item_data["location"] = item['location']
            print('Country: {}'.format(item['country']))
            item_data["country"] = item['country']
            item_data["viewItemURL"] = item["viewItemURL"]
            print('Condition: {}'.format(item['condition']))
            item_data["condition"] = item["condition"]['conditionDisplayName']
            item_data["price"] = item["sellingStatus"]["convertedCurrentPrice"]["value"]
            item_data["currency"] =item["sellingStatus"]["convertedCurrentPrice"]["_currencyId"]
            itm_cnt +=1
            #write_fridges_info(item_data)
            item_list.append(item_data)
            print("$"*30)
    except:
        if "errorMessage" in dictstr.keys() :
            if dictstr["errorMessage"]["error"]["errorId"] == '61' :
                print("@@"*20 + "Page Limit Reached!" + "@@"*20)
        break        
for item in item_list : 
    getExtraInfo(item)
    if item["capacity"]==None :
            extractCapacity(item)
    write_fridges_info(item)        
    display(item)

In [ ]:
Daily_Call